In [65]:
import pandas as pd
import re
import copy
import operator
import string
import nltk 
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree
from nltk.corpus import stopwords

In [2]:
gg = pd.read_json('gg2013.json', orient='records')
gg1 = pd.read_json('gg2015.json', orient='records')

In [3]:
copy = gg1.copy(deep=True)

In [4]:
def removePunc(x):
    x = re.sub(r'[@#]\w+', '', x) #taking out hashtags and @ 
    x = re.sub(r'(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)', '', x) #taking out links 
    x = re.sub(r'[!?\.,\'\":()]+', '', x) #taking out punctuation i.e ? ! . ' and " 
    x = re.sub(r'(RT|rt) ', '', x) #taking out the initial "RT "
    x= re.sub(r'(g|G)olden (g|G)lobes*', '', x)
    return x.strip()
# copy['text'] = copy['text'].str.lower()
copy['text'] = copy['text'].apply(lambda x: removePunc(str(x)))

In [5]:
#takes out duplicate retweets 
test = list(copy['text'].values)
print(len(test))
s = list(set(test))
print(len(s))
df = pd.DataFrame(s, columns=['text'])

1754153
671347


In [6]:
def findRows(x, contains, ncontains):
    s = "^"
    for word in contains:
        s += f'(?=.*{word})'
    for word in ncontains:
        s += f'(?!.*{word})'
    s += '.*$'
    if bool(re.match(s, x)):
        return True
    else:
        False
con = ['Best', 'wins', ]
con2 = ['Best']
noncon = []
df['Rows'] = df['text'].apply(lambda x: findRows(x, con, noncon))
ccc = df[~df['Rows'].isnull()]
print(len(ccc))

3984


In [8]:
# def findAwards2(text):
#     award = ""
#     right = text.split()
#     try:
#         ind = right.index('Best') #start at this index right so my first word will always be Bestttttt
#     except ValueError as ve: 
#         return ""
#     for i in range(ind, len(right)):
#         if award == "":
#                 award = right[i]
#         elif right[i] == 'for' or right[i] == 'at':
#             return award
#         else:
#             award = award + ' ' + right[i]
#     return award



In [21]:
def findAwards(text):
    award = ""
    firstsplit= text.split(' wins ', 1)
    if len(firstsplit) > 1:
        right = firstsplit[1].split()
        try:
            bestIndex = right.index('Best') #start at this index right so my first word will always be Bestttttt
        except ValueError as ve: 
            return ""
        for i in range(bestIndex, len(right)): 
            if award == "":
                award = right[i]
            elif right[i] == 'for' or right[i] == 'at':
                return award
            else:
                award = award + ' ' + right[i]
        award = award.lstrip()
    return award
    

def hashNgramsKai(dff):
    FINAL_NGRAMS = []
    NGRAMS_DICT = {}
    for i in range(len(dff)):

        j = findAwards(dff['text'].iloc[i])
        if j == "":
            continue
        if j in NGRAMS_DICT:
            NGRAMS_DICT[j] += 1
        else:
            NGRAMS_DICT[j] = 1
    cd = sorted(NGRAMS_DICT.items(),key=lambda x: (x[1], len(x[0])),reverse=True)
    return cd

In [115]:
lisTuples = hashNgramsKai(ccc)

In [80]:
cutoff = 5
def CUTOFF(tu, num):
    B = copy.deepcopy(tu)
    removes = []
    for i in B:
        if (i[1] < num):
            removes.append(i)
    for re in removes:
        B.remove(re)
    return B


In [116]:
filtered = CUTOFF(lisTuples, cutoff)

In [113]:
def substringfilter(dic):
    keys = list(dic.keys())
    numKeys = len(keys)
    toRemove = []
    for key in range(numKeys-1):
        for key2 in range(key+1, numKeys):
            if keys[key] in keys[key2]:
                toRemove.append(keys[key])
    for removeKey in toRemove:
        if removeKey in dic:
            del dic[removeKey]
    return dic

In [101]:
def Convert(tup):
    return dict(tup)
    
ddd = Convert(filtered)

In [114]:
substringfilter(ddd)

{'Best Actor in a Motion Picture Drama': 68,
 'Best Actor in a TV Series Drama': 61,
 'Best Supporting Actor in a Motion Picture': 38,
 'Best Screenplay': 29,
 'Best Motion Picture Musical or Comedy': 28,
 'Best TV Series Drama': 23,
 'Best Picture - Comedy': 22,
 'Best Drama': 22,
 'Best Motion Picture Comedy or Musical': 21,
 'Best Actor in a Drama': 20,
 'Best Actor Drama': 19,
 'Best Actor in a Motion Picture Comedy or Musical': 18,
 'Best Actor in a TV Drama': 18,
 'Best Actress in a Miniseries or TV Movie': 17,
 'Best motion picture - drama': 17,
 'Best Actor in a Comedy': 17,
 'Best Actor TV Drama': 15,
 'Best Actress Drama': 15,
 'Best Animated Film': 15,
 'Best Supporting Actor in a Series Miniseries or TV Movie': 14,
 'Best Supporting Actress in a Motion Picture': 14,
 'Best Actress in a Motion Picture Drama': 14,
 'Best Director Motion Picture': 14,
 'Best TV Comedy': 14,
 'Best Actress in a Motion Picture Comedy or Musical': 13,
 'Best Actress in a TV Series Drama': 13,
 'B

In [12]:
# text = '''
# Ben Affleck wins Best Director  AKA the In Your Face Matt Damon Award
# '''

# nltk_results = ne_chunk(pos_tag(word_tokenize(text)))
# for nltk_result in nltk_results:
#     print(nltk_result)
#     if type(nltk_result) != Tree and nltk_result[1] == 'NNP':
#         print(nltk_result)
#         name = ''
#         for nltk_result_leaf in nltk_result.leaves():
#             name += nltk_result_leaf[0] + ' '
#         print ('Type: ', nltk_result.label(), 'Name: ', name)

In [13]:
# def findNames(text):
#     names = []
#     firstsplit = text.split(' wins ', 1)
#     if len(firstsplit) > 1:
#         right = firstsplit[1].split()
#         for word in right:
#             if word[0].isupper():
#                 names.append(word)
#         return names
#     else:
#         return []
            
            
#     nltk_results = ne_chunk(pos_tag(word_tokenize(text)))
#     for nltk_result in nltk_results:
#         if type(nltk_result) != Tree and nltk_result[1] == 'NNP':
#             names.append(nltk_result[0])
#     return names
#             name = ''
#             for nltk_result_leaf in nltk_result.leaves():
#                 name += nltk_result_leaf[0] + ' '
# #             print ('Type: ', nltk_result.label(), 'Name: ', name)
#                 names.append(name)
# ccc['names'] = ccc['text'].apply(lambda x: findNames(x))
# ccc.head()

In [14]:
# def nGram(text):
#     text.reverse()
#     new_list = []
#     x = ''
#     for i in text:
#         x = i + ' ' + x
#         new_list.append(x)
#     new_list = [x.rstrip() for x in new_list]
#     return new_list


# def hashNgrams(dff):
#     FINAL_NGRAMS = []
#     NGRAMS_DICT = {}
#     for i in range(len(dff)):
#         l = nGram(dff['names'].iloc[i])
#         for j in l:
#             if j in NGRAMS_DICT:
#                 NGRAMS_DICT[j] += 1
#             else:
#                 NGRAMS_DICT[j] = 1
#     cd = sorted(NGRAMS_DICT.items(),key=lambda x: (x[1], len(x[0])),reverse=True)
#     return cd

In [15]:
# hashNgrams(ccc)

In [16]:
# def what(text):
#     tweet = text.split()
#     ind = tweet.index('Best')
#     if len(tweet) > 1:
        

In [17]:
# def nGram(text):
#     test = text
#     firstsplit = test.split('Best', 1)
#     if len(firstsplit) > 1:
#         left = firstsplit[0].split(' ')
#         left.reverse()
#         right = firstsplit[1].split(' ')
#         new_list = []
#         x = ""
#         for i in left:
#             x =  i + x + ' '
#             new_list.append(x)
#         x = ""
#         for i in right:
#             x = x + i + ' '
#             new_list.append(x)
#         new_list = [x.rstrip() for x in new_list]
#     else:
#         new_list = []
#     return new_list

In [18]:
# FINAL_NGRAMS = []
# NGRAMS_DICT = {}
# for i in range(len(df)):
    
#     l = nGram(df['text'].iloc[i])
#     for j in l:
#         if j in NGRAMS_DICT:
#             NGRAMS_DICT[j] += 1
#         else:
#             NGRAMS_DICT[j] = 1
# cd = sorted(NGRAMS_DICT.items(),key=operator.itemgetter(1),reverse=True)
# cd